### Проверка работоспособности и качества пайплайна

Проверяем работу предобученной модели на отложенной (валидационной) выборке

In [1]:
import numpy as np
import pandas as pd
from sklearn.metrics import roc_auc_score
from pathlib import Path
import dill

In [2]:
DATA_ROOT = Path('./data2/')
MODELS_PATH = Path('./models2/')

# valid dataset path
VALID_DATASET_PATH = DATA_ROOT / 'df_valid.csv'

# model path
MODEL_FILE_PATH = MODELS_PATH / 'model.pkl'

In [3]:
df_valid = pd.read_csv(VALID_DATASET_PATH, sep=';')

df_valid.head(8)

,customer_id,tr_datetime,mcc_code,tr_type,amount,term_id,gender
0,79780256,37 13:36:14,4814,1030,-3144.28,NaN,1.0
1,79780256,39 10:16:49,4814,1030,-5614.79,NaN,1.0
2,79780256,44 09:41:33,6011,2010,-112295.79,NaN,1.0
3,79780256,44 09:42:44,6011,2010,-67377.47,NaN,1.0
4,79780256,51 08:53:56,4814,1030,-1122.96,NaN,1.0
5,79780256,51 08:55:09,4814,1030,-2245.92,NaN,1.0
6,79780256,58 11:18:31,6011,2010,-67377.47,NaN,1.0
7,79780256,59 12:29:60,6011,2010,-22459.16,NaN,1.0


In [4]:
users = df_valid.groupby('customer_id')['mcc_code'].apply(list).reset_index()
types = df_valid.groupby('customer_id')['tr_type'].apply(list).reset_index()
genders = df_valid.groupby('customer_id')['gender'].mean().reset_index()

df_valid = pd.merge(users, types,  on='customer_id', how='left')
df_valid = pd.merge(df_valid, genders,  on='customer_id', how='left')

df_valid

,customer_id,mcc_code,tr_type,gender
0,42096,"[5411, 4814, 6011, 6011, 5411, 7311, 5499, 481...","[1010, 1030, 4011, 2011, 1010, 1100, 1110, 103...",0.0
1,104032,"[4814, 4814, 4814, 6011, 4814, 4814, 4814, 481...","[1030, 1030, 1030, 2010, 1030, 1030, 1030, 103...",1.0
2,353313,"[4814, 4814, 4814, 4814, 4814, 4814, 4814, 601...","[1030, 1030, 1030, 1030, 1030, 1030, 1030, 201...",0.0
3,402290,"[6011, 6011, 4814, 6011, 6011, 6011, 6011, 601...","[4010, 2010, 1030, 2010, 2010, 4010, 2010, 401...",0.0
4,529117,"[6011, 6011, 5411, 6011, 5411, 6011, 6011, 541...","[2010, 2010, 1010, 2010, 1010, 2010, 2010, 111...",0.0
...,...,...,...,...
835,99468407,"[4814, 4814, 5499, 4814, 6011, 4814, 5499, 549...","[1030, 1030, 1010, 1030, 2010, 1030, 1010, 101...",1.0
836,99568688,"[5735, 7994, 7512, 7512, 4814, 4814, 6010, 601...","[1200, 1200, 1210, 6200, 1030, 1030, 2020, 202...",1.0
837,99848367,"[4814, 4814, 6011, 6011, 6011, 6010, 6011, 601...","[1030, 1030, 2010, 2010, 2010, 7070, 2010, 701...",0.0
838,99864691,"[6011, 5541, 6011, 6010, 4812, 6011, 4814, 601...","[7010, 1010, 2010, 7030, 1110, 7010, 1030, 701...",1.0


In [5]:
X_valid = df_valid.drop(columns='gender')
y_valid = df_valid['gender']

In [6]:
with open(MODEL_FILE_PATH, 'rb') as model:
    model = dill.load(model)

In [7]:
y_score = model.predict_proba(X_valid)[:, 1]

In [9]:
roc_auc_score(y_valid, y_score)

0.815505267579215

In [10]:
y_score

array([0.18028777, 0.54277587, 0.42091361, 0.67005882, 0.42419603,
       0.43180624, 0.28117141, 0.62307411, 0.45111766, 0.26025409,
       0.76240149, 0.16428788, 0.2215534 , 0.16478835, 0.30978299,
       0.40618164, 0.33787052, 0.39636386, 0.11175273, 0.05188466,
       0.38692482, 0.72329994, 0.17957286, 0.16782719, 0.58291923,
       0.4119136 , 0.18734306, 0.52804419, 0.96028578, 0.88895489,
       0.69345723, 0.42327309, 0.62359443, 0.88322772, 0.23171817,
       0.8333193 , 0.20017899, 0.85137127, 0.70504888, 0.09157389,
       0.38281507, 0.39791166, 0.51953073, 0.0801551 , 0.34184678,
       0.51065861, 0.08581696, 0.45985252, 0.50839961, 0.41289657,
       0.16267674, 0.89603214, 0.4040318 , 0.29985514, 0.49259819,
       0.49106911, 0.92248753, 0.52012901, 0.964647  , 0.02264219,
       0.58646164, 0.4080198 , 0.13376105, 0.16021001, 0.54455313,
       0.64054809, 0.13388987, 0.53357533, 0.6901067 , 0.08744183,
       0.25445238, 0.77836136, 0.00765205, 0.77065691, 0.40446